<a href="https://colab.research.google.com/github/sindhu819/Spark-2020/blob/main/Copy_of_classification_by_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Running Pyspark in Colab**

To run spark in Colab, we need to first install pyspark


In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [75.1 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 k

Congrats! Your Colab is ready to run Pyspark. 

# EDA

The goal of this exercise to explore and analyze the Boston Housing dataset.   Load the dataset into your Colab directory from your local system:

In [ ]:
#1 Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#2. Get the file
#make sure you upload all your data files to your Google drive and change share->Advanced->change->anyone with the link can view
downloaded = drive.CreateFile({'id':'1o8OH-hFyZxaCjxlSb2AtopjVaMdVmCLL'}) # replace the id with id of file you want to access
downloaded.GetContentFile('iris.csv')

## load iris data

In [ ]:
iris = spark.read.csv('iris.csv', header=True, inferSchema=True)

In [ ]:
iris.show(5)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [ ]:
iris.dtypes

[('sepal_length', 'double'),
 ('sepal_width', 'double'),
 ('petal_length', 'double'),
 ('petal_width', 'double'),
 ('species', 'string')]

In [ ]:
iris.describe().show()

+-------+------------------+-------------------+------------------+------------------+---------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  species|
+-------+------------------+-------------------+------------------+------------------+---------+
|  count|               150|                150|               150|               150|      150|
|   mean| 5.843333333333335| 3.0540000000000007|3.7586666666666693|1.1986666666666672|     null|
| stddev|0.8280661279778637|0.43359431136217375| 1.764420419952262|0.7631607417008414|     null|
|    min|               4.3|                2.0|               1.0|               0.1|   setosa|
|    max|               7.9|                4.4|               6.9|               2.5|virginica|
+-------+------------------+-------------------+------------------+------------------+---------+



## Merge features to create a features column

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row

In [ ]:
iris2 = iris.rdd.map(lambda x: Row(features=Vectors.dense(x[:-1]), species=x[-1])).toDF()
iris2.show(5)

+-----------------+-------+
|         features|species|
+-----------------+-------+
|[5.1,3.5,1.4,0.2]| setosa|
|[4.9,3.0,1.4,0.2]| setosa|
|[4.7,3.2,1.3,0.2]| setosa|
|[4.6,3.1,1.5,0.2]| setosa|
|[5.0,3.6,1.4,0.2]| setosa|
+-----------------+-------+
only showing top 5 rows



In [ ]:
iris2.groupBy('species').count().orderBy('species').show()

+----------+-----+
|   species|count|
+----------+-----+
|    setosa|   50|
|versicolor|   50|
| virginica|   50|
+----------+-----+



In [ ]:
iris2.select("species").distinct().count()

3

In [ ]:
iris2.createOrReplaceTempView("iris")

In [ ]:
spark.sql("select species,count(*) from iris group by species order by species").show()

+----------+--------+
|   species|count(1)|
+----------+--------+
|    setosa|      50|
|versicolor|      50|
| virginica|      50|
+----------+--------+



## Index label column with `StringIndexer`

### Import libraries

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

### Build pipeline
Let's change the categorical species to numeric labels.

In [ ]:
stringindexer = StringIndexer(inputCol='species', outputCol='label')

### Transform data

In [ ]:
iris_df = stringindexer.fit(iris2).transform(iris2)
iris_df.show(5)

+-----------------+-------+-----+
|         features|species|label|
+-----------------+-------+-----+
|[5.1,3.5,1.4,0.2]| setosa|  2.0|
|[4.9,3.0,1.4,0.2]| setosa|  2.0|
|[4.7,3.2,1.3,0.2]| setosa|  2.0|
|[4.6,3.1,1.5,0.2]| setosa|  2.0|
|[5.0,3.6,1.4,0.2]| setosa|  2.0|
+-----------------+-------+-----+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import rand
iris_df.orderBy(rand()).limit(10).show()

+-----------------+----------+-----+
|         features|   species|label|
+-----------------+----------+-----+
|[6.7,3.3,5.7,2.5]| virginica|  1.0|
|[7.7,2.8,6.7,2.0]| virginica|  1.0|
|[5.8,2.7,4.1,1.0]|versicolor|  0.0|
|[4.9,3.1,1.5,0.1]|    setosa|  2.0|
|[6.1,2.9,4.7,1.4]|versicolor|  0.0|
|[5.8,4.0,1.2,0.2]|    setosa|  2.0|
|[6.8,3.2,5.9,2.3]| virginica|  1.0|
|[6.1,2.6,5.6,1.4]| virginica|  1.0|
|[4.6,3.6,1.0,0.2]|    setosa|  2.0|
|[6.4,2.8,5.6,2.2]| virginica|  1.0|
+-----------------+----------+-----+



### Check the data one more time

In [ ]:
iris_df.describe().show(5)

+-------+---------+------------------+
|summary|  species|             label|
+-------+---------+------------------+
|  count|      150|               150|
|   mean|     null|               1.0|
| stddev|     null|0.8192319205190403|
|    min|   setosa|               0.0|
|    max|virginica|               2.0|
+-------+---------+------------------+



In [ ]:
iris_df.dtypes

[('features', 'vector'), ('species', 'string'), ('label', 'double')]

## Naive Bayes classification

### Split data into training and test sets

In [ ]:
train, test = iris_df.randomSplit([0.8, 0.2], seed=1234)

### Build cross-validation model

#### Estimator

In [ ]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(featuresCol="features", labelCol="label",smoothing=1.0, modelType="multinomial")
nb = nb.fit(train)
pred = nb.transform(test)
pred.select('species','label','prediction').show()

+----------+-----+----------+
|   species|label|prediction|
+----------+-----+----------+
|    setosa|  2.0|       2.0|
|    setosa|  2.0|       2.0|
|    setosa|  2.0|       2.0|
|    setosa|  2.0|       2.0|
|    setosa|  2.0|       2.0|
|    setosa|  2.0|       2.0|
|    setosa|  2.0|       2.0|
|    setosa|  2.0|       2.0|
|versicolor|  0.0|       0.0|
|    setosa|  2.0|       2.0|
|    setosa|  2.0|       2.0|
|versicolor|  0.0|       0.0|
| virginica|  1.0|       1.0|
|versicolor|  0.0|       0.0|
|versicolor|  0.0|       0.0|
|versicolor|  0.0|       0.0|
| virginica|  1.0|       1.0|
|versicolor|  0.0|       0.0|
| virginica|  1.0|       0.0|
| virginica|  1.0|       1.0|
+----------+-----+----------+
only showing top 20 rows



#### Prediction accurary
Four accuracy matrices are avaiable for this evaluator. 
* `f1`
* `weightedPrecision`
* `weightedRecall`
* `accuracy`

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

In [ ]:
# Accuracy
acc = eval.evaluate(pred, {eval.metricName: "accuracy"})
# acc = eval.setMetricName('accuracy').evaluate(pred)# another way to evaluate
print("Accuracy: %.3f" % acc)

# Weighted Precision
# weightedPrecision = eval.evaluate(pred, {eval.metricName: "weightedPrecision"})
weightedPrecision = eval.setMetricName('weightedPrecision').evaluate(pred)
print("Weighted Precision: %.3f" % weightedPrecision)

# weightedRecall 
# weightedRecall = eval.evaluate(pred, {eval.metricName: "weightedRecall"})
weightedRecall= eval.setMetricName('weightedRecall').evaluate(pred)
print("Weighted Recall: %.3f" % weightedRecall)

# f1
# f1 = eval.evaluate(pred, {eval.metricName: "f1"})
f1=eval.setMetricName('f1').evaluate(pred)
print("f1: %.3f" %f1)

Accuracy: 0.958
Weighted Precision: 0.964
Weighted Recall: 0.958
f1: 0.958


### Confusion matrix

In [ ]:
conf_mat = pred.select('label', 'prediction')
conf_mat.rdd.zipWithIndex().countByKey()

defaultdict(int,
            {Row(label=0.0, prediction=0.0): 7,
             Row(label=1.0, prediction=0.0): 1,
             Row(label=1.0, prediction=1.0): 6,
             Row(label=2.0, prediction=2.0): 10})

## Use MulticlassMetrics library under mllib ##   
### Convert to RDD before using mllib ###

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics
predictionRDD = pred.select(['label', 'prediction']) \
                            .rdd.map(lambda line: (line[1], line[0]))
metrics = MulticlassMetrics(predictionRDD)

In [ ]:
# Confusion Matrix
cm = metrics.confusionMatrix().toArray()
print("Confusion Matrix:")
print("%10s %12s %12s %12s" % ("real\pred" ,"predicted 0", "predicted 1", "predicted 2"))
for i in range (0,3):
  print("real %2s" % i, end='')
  for j in range (0,3):
    print("%12d" % cm[i][j], end='')
  print()

Confusion Matrix:
 real\pred  predicted 0  predicted 1  predicted 2
real  0           7           0           0
real  1           1           6           0
real  2           0           0          10


In [ ]:
# Define a function to evaluate each of the three labels, i.e., 0, 1, and 2
def cr(label_in):
  precision = metrics.precision(label=label_in)
  recall = metrics.recall(label=label_in)
  F1_Measure = metrics.fMeasure(label=label_in)
  support = test.filter(test.label==label_in).count()
  print("%10s %12.2f  %12.2f %12.2f %12d" % \
          (label_in,precision, recall, F1_Measure, support))

In [ ]:
import numpy as np
print("            Classification Report")
print("%10s %12s  %12s %12s %12s"    % ("label","precision","recall","f1-score","support"))
for i in np.arange(0.0, 3.0, 1.0):
  cr(i)

            Classification Report
     label    precision        recall     f1-score      support
       0.0         0.88          1.00         0.93            7
       1.0         1.00          0.86         0.92            7
       2.0         1.00          1.00         1.00           10


## Extend the process with Grid search and crossover

### Build cross-validation model

#### Estimator

In [ ]:
from pyspark.ml.classification import NaiveBayes
naivebayes = NaiveBayes(featuresCol="features", labelCol="label")

#### Parameter grid

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder().\
    addGrid(naivebayes.smoothing, [0, 1, 2, 4, 8]).\
    build()

#### Evaluator
There are three categories in the label column. Therefore, we use `MulticlassClassificationEvaluator`

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator()

#### Build cross-validation model

In [ ]:
from pyspark.ml.tuning import CrossValidator
crossvalidator = CrossValidator(estimator=naivebayes, estimatorParamMaps=param_grid, evaluator=evaluator)

#### Fit cross-validation model

In [ ]:
crossvalidation_mode = crossvalidator.fit(train)

#### Prediction on training and test sets

In [ ]:
pred_train = crossvalidation_mode.transform(train)
pred_train.show(5)

+-----------------+-------+-----+--------------------+--------------------+----------+
|         features|species|label|       rawPrediction|         probability|prediction|
+-----------------+-------+-----+--------------------+--------------------+----------+
|[4.4,3.2,1.3,0.2]| setosa|  2.0|[-12.291171378989...|[0.19555929123370...|       2.0|
|[4.5,2.3,1.3,0.3]| setosa|  2.0|[-11.142786320680...|[0.26949095780384...|       2.0|
|[4.6,3.1,1.5,0.2]| setosa|  2.0|[-12.550742246764...|[0.21656037289160...|       2.0|
|[4.6,3.2,1.4,0.2]| setosa|  2.0|[-12.592365024690...|[0.20052392100539...|       2.0|
|[4.6,3.4,1.4,0.3]| setosa|  2.0|[-13.148463542368...|[0.19824085251800...|       2.0|
+-----------------+-------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
pred_test = crossvalidation_mode.transform(test)
pred_test.show(5)

+-----------------+-------+-----+--------------------+--------------------+----------+
|         features|species|label|       rawPrediction|         probability|prediction|
+-----------------+-------+-----+--------------------+--------------------+----------+
|[4.3,3.0,1.1,0.1]| setosa|  2.0|[-11.402228667810...|[0.18240945200010...|       2.0|
|[4.4,2.9,1.4,0.2]| setosa|  2.0|[-11.923306551010...|[0.22553075812617...|       2.0|
|[4.4,3.0,1.3,0.2]| setosa|  2.0|[-11.964929328937...|[0.20930932928504...|       2.0|
|[4.8,3.1,1.6,0.2]| setosa|  2.0|[-12.851935892465...|[0.22180002940740...|       2.0|
|[5.0,3.3,1.4,0.2]| setosa|  2.0|[-13.114876846919...|[0.18601727933390...|       2.0|
+-----------------+-------+-----+--------------------+--------------------+----------+
only showing top 5 rows



#### Best model from cross validation

In [ ]:
print("The parameter smoothing has best value:",
      crossvalidation_mode.bestModel._java_obj.getSmoothing())

The parameter smoothing has best value: 8.0


#### Prediction accurary
Four accuracy matrices are avaiable for this evaluator. 
* `f1`
* `weightedPrecision`
* `weightedRecall`
* `accuracy`

##### Prediction accuracy on training data

In [ ]:
print('training data (f1):', evaluator.setMetricName('f1').evaluate(pred_train), "\n",
     'training data (weightedPrecision): ', evaluator.setMetricName('weightedPrecision').evaluate(pred_train),"\n",
     'training data (weightedRecall): ', evaluator.setMetricName('weightedRecall').evaluate(pred_train),"\n",
     'training data (accuracy): ', evaluator.setMetricName('accuracy').evaluate(pred_train))



training data (f1): 0.968236789665361 
 training data (weightedPrecision):  0.9689250225835592 
 training data (weightedRecall):  0.9682539682539681 
 training data (accuracy):  0.9682539682539683


##### Prediction accuracy on test data

In [ ]:
print('test data (f1):', evaluator.setMetricName('f1').evaluate(pred_test), "\n",
     'test data (weightedPrecision): ', evaluator.setMetricName('weightedPrecision').evaluate(pred_test),"\n",
     'test data (weightedRecall): ', evaluator.setMetricName('weightedRecall').evaluate(pred_test),"\n",
     'test data (accuracy): ', evaluator.setMetricName('accuracy').evaluate(pred_test))

test data (f1): 0.958119658119658 
 test data (weightedPrecision):  0.9635416666666667 
 test data (weightedRecall):  0.9583333333333334 
 test data (accuracy):  0.9583333333333334


### Confusion matrix

#### Confusion matrix on training data

In [ ]:
train_conf_mat = pred_train.select('label', 'prediction')
train_conf_mat.rdd.zipWithIndex().countByKey()

defaultdict(int,
            {Row(label=0.0, prediction=0.0): 42,
             Row(label=0.0, prediction=1.0): 1,
             Row(label=1.0, prediction=0.0): 3,
             Row(label=1.0, prediction=1.0): 40,
             Row(label=2.0, prediction=2.0): 40})

#### Confusion matrix on test data

In [ ]:
test_conf_mat = pred_test.select('label', 'prediction')
test_conf_mat.rdd.zipWithIndex().countByKey()

defaultdict(int,
            {Row(label=0.0, prediction=0.0): 7,
             Row(label=1.0, prediction=0.0): 1,
             Row(label=1.0, prediction=1.0): 6,
             Row(label=2.0, prediction=2.0): 10})

**From the confusion matrices on both training and test data, we can see that there are only a few mismatches between prediction and label values.**